In [38]:
import pandas as pd
import quandl, math, datetime
import numpy as np
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

In [2]:
df = quandl.get('WIKI/GOOGL')

In [4]:
print(df.head())

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [6]:
df = df[['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume',]]

In [9]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100.0

C:\Users\user\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df['PCT_Change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100.0

C:\Users\user\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df = df[['Adj. Close','HL_PCT','PCT_Change','Adj. Volume']]

In [12]:
print(df.head())

            Adj. Close    HL_PCT  PCT_Change  Adj. Volume
Date                                                     
2004-08-19   50.322842  3.712563    0.324968   44659000.0
2004-08-20   54.322689  0.710922    7.227007   22834300.0
2004-08-23   54.869377  3.729433   -1.227880   18256100.0
2004-08-24   52.597363  6.417469   -5.726357   15247300.0
2004-08-25   53.164113  1.886792    1.183658    9188600.0


In [29]:
forcast_col = 'Adj. Close'
df.fillna(-99999, inplace = True)
forcast_out = int(math.ceil(0.01*len(df)))


In [30]:
df['label'] = df[forcast_col].shift(-forcast_out)
df.dropna(inplace=True)
print(df.head())

            Adj. Close    HL_PCT  PCT_Change  Adj. Volume      label
Date                                                                
2004-08-19   50.322842  3.712563    0.324968   44659000.0  65.742942
2004-08-20   54.322689  0.710922    7.227007   22834300.0  65.000651
2004-08-23   54.869377  3.729433   -1.227880   18256100.0  66.495265
2004-08-24   52.597363  6.417469   -5.726357   15247300.0  67.739104
2004-08-25   53.164113  1.886792    1.183658    9188600.0  69.399229


In [33]:
X = np.array(df.drop(['label'],1))
y = np.array(df['label'])
X = preprocessing.scale(X)


In [34]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y, test_size = 0.2)
clf = LinearRegression()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)

In [37]:
print(accuracy)
print(forcast_out)

0.957733892978
28


In [39]:
X = X[: -forcast_out]
X_lately = X[-forcast_out :]
forcast_set = clf.predict(X_lately)
print(forcast_set, accuracy, forcast_out)

(array([ 570.43008604,  573.66143638,  557.68958478,  553.6511964 ,
        561.29377402,  550.26277369,  560.87947237,  557.49426288,
        565.47608616,  563.33968098,  564.74688842,  564.44884583,
        573.93881242,  563.33438338,  562.7197489 ,  556.38693657,
        561.57290493,  552.99274672,  547.63563895,  540.60524519,
        544.03805267,  544.22625457,  548.73085366,  548.93494889,
        549.55702142,  547.88113018,  539.1887194 ,  541.01039115]), 0.95773389297825096, 28)
